In [5]:
import os
import time
import cv2
import scipy.io as sio
import numpy as np
import pandas as pd
import pydicom as dcm
import plotly.express as px

from cest_mrf.metrics.dot_product_pickle import dot_prod_matching

from my_funcs.plot_functions import t1_t2_pixel_reader
from my_funcs.plot_functions import find_n

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from my_funcs.path_functions import make_folder

from skimage.morphology import binary_erosion, binary_dilation
from skimage import measure
from skimage.measure import label, regionprops, regionprops_table



## 1.1. Enter Data: ##

In [6]:
# Root stats:
# general_fn = os.path.abspath(os.curdir)
current_dir = os.getcwd()  # Get the current directory
parent_dir = os.path.dirname(current_dir)  # Navigate up one directory level
general_fn = os.path.join(parent_dir, 'data')

txt_file_name = 'labarchive_notes.txt'
save_name = 'mouse_mt_107'
fp_prtcl_names = ['107a']  # to be looped later with all options # 107a , 51_glu
f_const = 3 / 110000
dict_ranges =  {'fs_0': (-1*f_const, 30*f_const),
                'ksw_0': (8400, 12000),
                't1w': (1.2, 2.4),
                't2w': (0.03, 0.12)}

subject_dicts = [
    # {'scan_name': '24_04_04_glu_mouse_37deg',
    #  'sub_name': '1_mouse1_left',
    #  'month': 'apr',
    #  'save_name': save_name,
    #  'center': [39, 15],
    #  'voxel_origin': [28, 13],
    #  'temp': 37,
    #  'z_b1s': [0.7, 1.5],
    #  'z_b1s_names': ['0p7uT', '1p5uT']
    #  },
    # {'scan_name': '24_04_04_glu_mouse_37deg',
    #  'sub_name': '2_mouse2_right',
    #  'month': 'apr',
    #  'center': [25, 14],
    #  'voxel_origin': [16, 11],
    #  'save_name': save_name,
    #  'temp': 37,
    #  'z_b1s': [0.7, 1.5],
    #  'z_b1s_names': ['0p7uT', '1p5uT']
    #  },
        {'scan_name': '24_04_04_glu_mouse_37deg',
     'sub_name': '3_mouse3_two_ears',
     'month': 'apr',
     'center': [30, 19],
     'voxel_origin': [21, 14],
     'save_name': save_name,
     'temp': 37,
     'z_b1s': [0.7, 1.5],
     'z_b1s_names': ['0p7uT', '1p5uT']
     },
     #    {'scan_name': '24_01_28_glu_mouse_37deg',
     # 'sub_name': '1_or_dino_mouse2_right_ear_middle_pierce',
     # 'month': 'jan',
     # 'center': [0, 0],
     # 'voxel_origin': [0, 0],
     # 'save_name': save_name,
     # 'temp': 37,
     # 'z_b1s': [0.7, 2],
     # 'z_b1s_names': ['0p7uT', '2uT']
     # },
 ]


## 1.2. Filter dict & Run Dot Product ##

In [7]:
# # load un-filtered dict
# for fp_prtcl_name in fp_prtcl_names:
#     dict_fn = os.path.join('exp', save_name, fp_prtcl_name, 'dict.csv')
#     # synt_df = pd.read_csv(dict_fn, header=0)
#     # 
#     # # filter dict using dict_range
#     # f_dict_fn = os.path.join('exp', save_name, fp_prtcl_name, 'f_dict.csv')
#     # df_masks = [synt_df[column].between(min_val, max_val) for column, (min_val, max_val) in dict_ranges.items()]
#     # filtered_df = synt_df[np.all(df_masks, axis=0)]
#     # filtered_df.to_csv(path_or_buf=f_dict_fn, index=False)  # To save as CSV without row indices
# 
#     for subject_i, subject_dict in enumerate(subject_dicts):
#         start = time.perf_counter()
#         print(f'################################# start of phantom {subject_i+1} #################################')
#         glu_phantom_fn = os.path.join(general_fn, 'scans', subject_dict['scan_name'],
#                                  subject_dict['sub_name'])
#         # val_range = subject_dict['dict_ranges']
# 
#         glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_prtcl_name)
#         glu_acquired_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
# 
#         # create acquired data folder: root->scans->date->subject->E->mrf_files->acquired_data
#         acquired_data_fn = os.path.join(glu_phantom_mrf_files_fn, 'acquired_data.mat')
#         make_folder(glu_phantom_mrf_files_fn)
# 
#         # save acquired data to: root->scans->date->subject->E->mrf_files->acquired_data
#         sio.savemat(acquired_data_fn, {'acquired_data': glu_acquired_data})
#         print(f'Acquired data was saved as {glu_acquired_data.shape} sized array')
# 
#         quant_maps = dot_prod_matching(dict_fn=dict_fn, acquired_data_fn=acquired_data_fn)        
#         # quant_maps = euc_dist_matching(dict_fn=f_dict_fn, acquired_data_fn=acquired_data_fn)
#         end = time.perf_counter()
#         s = (end - start)
#         print(f"Dot product matching took {s:.03f} s.")
# 
#         # save acquired data to: root->scans->date->subject->E->mrf_files->quant_maps.mat
#         quant_maps_fn = os.path.join(glu_phantom_mrf_files_fn, 'quant_maps.mat')
#         sio.savemat(quant_maps_fn, quant_maps)
#         quant_maps_fn = os.path.join('images', save_name, f'subject_{subject_i+1}', 'quant_maps.mat')
#         sio.savemat(quant_maps_fn, quant_maps)
#         print('quant_maps.mat saved')
# 
#         print(f'#################################  end of phantom {subject_i+1}  #################################')


In [11]:
# MT map from 52 protocol (check!)
fp_mt_prtcl_name = 'MT52'
dict_fn = os.path.join(general_fn, 'exp', 'MT_pickle', fp_mt_prtcl_name, 'dict.pkl')

for subject_i, subject_dict in enumerate(subject_dicts):
    start = time.perf_counter()
    print(f'################################# start of phantom {subject_i+1} #################################')
    glu_phantom_fn = os.path.join(general_fn, 'scans', subject_dict['scan_name'],
                             subject_dict['sub_name'])

    glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_mt_prtcl_name)
    glu_acquired_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)

    # create acquired data folder: root->scans->date->subject->E->mrf_files->acquired_data
    acquired_data_fn = os.path.join(glu_phantom_mrf_files_fn, 'acquired_data.mat')
    make_folder(glu_phantom_mrf_files_fn)

    # save acquired data to: root->scans->date->subject->E->mrf_files->acquired_data
    sio.savemat(acquired_data_fn, {'acquired_data': glu_acquired_data})
    print(f'Acquired data was saved as {glu_acquired_data.shape} sized array')

    quant_maps = dot_prod_matching(dict_fn=dict_fn, acquired_data_fn=acquired_data_fn)
    end = time.perf_counter()
    s = (end - start)
    print(f"Dot product matching took {s:.03f} s.")

    # save acquired data to: root->scans->date->subject->E->mrf_files->quant_maps.mat
    quant_maps_fn = os.path.join('images', save_name, f'subject_{subject_i+1}', 'mt_quant_maps.mat')
    sio.savemat(quant_maps_fn, quant_maps)
    print('quant_maps.mat saved')

    print(f'#################################  end of phantom {subject_i+1}  #################################')


################################# start of phantom 1 #################################
Folder 'D:\mrf\data\scans\24_04_04_glu_mouse_37deg\3_mouse3_two_ears\12\mrf_files' already exists.
Acquired data was saved as (31, 40, 64) sized array
Dot product matching took 0.783 s.
quant_maps.mat saved
#################################  end of phantom 1  #################################


## subject/protocol choice ##

In [15]:
fp_prtcl_name = fp_prtcl_names[0]
subject_i = 2

subject_dict = subject_dicts[subject_i]
phantom_choice = subject_i+1
glu_mouse_fn = os.path.join(general_fn, 'scans', subject_dict['scan_name'],
                          subject_dict['sub_name'])
glu_mouse_dicom_fn, glu_mouse_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_mouse_fn, txt_file_name, fp_prtcl_name)


## Segmentation ##

In [16]:
from skimage import filters, measure
from skimage import img_as_ubyte

r_c, c_c = subject_dict['center']
r_d, c_d = 18, 14

"""cest loading:"""
cest_data = bruker_dataset.data[r_c-r_d:r_c+r_d,c_c-c_d:c_c+c_d,0,:]
count = np.shape(cest_data)[-1]  # # of images

"""highest contrast image finding:"""
# Compute RMS contrast for each image
rms_contrast = np.zeros(count)
for i in range(count):
    rms_contrast[i] = np.std(cest_data[:, :, i])

# Find index of image with highest contrast
index = np.argmax(rms_contrast)
contrasted_image = np.transpose(np.uint16(cest_data[:, :, index]))

"""brain locating:"""
edges = filters.sobel(contrasted_image)
edges = (edges / edges.max())*255
# normalized_array_uint8 = img_as_ubyte(edges)
edges = np.uint16(edges)

px.imshow(bruker_dataset.data[:,:,0,0].T)


In [17]:
# import numpy as np
# import matplotlib.pyplot as plt
# 
# class InteractiveMask:
#     def __init__(self, image):
#         self.image = image
#         self.mask = np.zeros_like(image, dtype=bool)
#         self.fig, self.ax = plt.subplots()
#         self.ax.imshow(image)
#         self.fig.canvas.mpl_connect('button_press_event', self.on_click)
#         self.fig.canvas.mpl_connect('key_press_event', self.on_key_press)
#         self.current_x = None
#         self.current_y = None
#         self.update_plot()
#         plt.show()
# 
#     def on_click(self, event):
#         if event.button == 1 and event.inaxes:
#             self.current_x = int(event.xdata + 0.5)
#             self.current_y = int(event.ydata + 0.5)
#             self.mask[self.current_y, self.current_x] = ~self.mask[self.current_y, self.current_x]
#             self.update_plot()
# 
#     def on_key_press(self, event):
#         if event.key == 'enter':
#             plt.close()
# 
#     def update_plot(self):
#         self.ax.clear()
#         self.ax.imshow(self.image, cmap='gray')
#         self.ax.imshow(self.mask, cmap='jet', alpha=0.5)
#         plt.draw()
# 
#     def get_mask(self):
#         return self.mask
# 
# # Example usage:
# %matplotlib notebook
# mask_creator = InteractiveMask(contrasted_image)


In [18]:
# brain_mask = np.zeros_like(bruker_dataset.data[:,:,0,0])
# 
# outline_image = np.uint16(mask_creator.get_mask())
# image_binary = outline_image < 0.5
# label_image = label(image_binary)
# 
# px.imshow(label_image)
# # Find the brain region
# for region in regionprops(label_image):
#     # print(region.area)
#     # print(region.area_filled)
#     # print(region.axis_minor_length / region.axis_major_length)
#     if 200 < region.area_filled < 500:
#         brain_label = region.label
#         brain_region = region
# 
# # Create a mask for each circle and set the corresponding pixels to one
# mask = np.zeros_like(contrasted_image)
# brain_mask[r_c-r_d:r_c+r_d,c_c-c_d:c_c+c_d] = (label_image == brain_label).T
# 
# mask_path = os.path.join(os.path.dirname(os.path.dirname(glu_mouse_mrf_files_fn)), 'mask.npy')
# np.save(mask_path, brain_mask.T)


In [19]:
mask_path = os.path.join(os.path.dirname(os.path.dirname(glu_mouse_mrf_files_fn)), 'mask.npy')
mask = np.load(mask_path)
subject_dict['roi_mask'] = mask

c_o, r_o = subject_dict['voxel_origin']
l = 5
voxel_mask = np.zeros_like(mask)
voxel_mask[r_o:r_o+l, c_o:c_o+l] = 1
subject_dict['roi_voxel_mask'] = voxel_mask
px.imshow(mask - voxel_mask)


## t1, t2 maps ##

In [20]:
t1_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1', image_file=4)
t2_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')

custom_viridis = np.array(plotly.colors.sequential.Viridis)
custom_plasma = np.array(plotly.colors.sequential.Plasma)

# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.085, subplot_titles=['t1 [ms]', 't2 [ms]', 'Dot Product'])

# Add heatmaps for the three arrays
heatmap_fs = go.Heatmap(z=t1_pixels*mask, colorscale=custom_viridis, coloraxis='coloraxis1')
heatmap_ksw = go.Heatmap(z=t2_pixels*mask, colorscale=custom_plasma, coloraxis='coloraxis2')

fig.add_trace(heatmap_fs, row=1, col=1)
fig.add_trace(heatmap_ksw, row=1, col=2)

fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"Phantom {subject_dict['month']} {subject_dict['temp']}°C: '{fp_prtcl_name}' t1 & t2 maps",
    showlegend=False,  # Hide legend
    height=345,
    width=900,  # Set a width based on your preference
    margin=dict(l=10, r=40, t=60, b=20),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)

# Add individual titles and separate colorbars
for i, title in enumerate(['Glu [mM]', 'ksw [Hz]'], start=1):
    fig.update_xaxes(row=1, col=i, showgrid=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, row=1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# colorbar_fs = {'colorscale': custom_viridis, 'cmin': 1000, 'cmax': 1500}
colorbar_fs = {'colorscale': custom_viridis, 'cmin': 1300, 'cmax': 2300}
colorbar_ksw = {'colorscale': custom_plasma, 'cmin': 40, 'cmax': 110}

fig.update_layout(
    coloraxis1=colorbar_fs,
    coloraxis2=colorbar_ksw,
    coloraxis_colorbar=dict(x=0.46, y=0.5),
    coloraxis2_colorbar=dict(x=1, y=0.5),
)

# Show the plot
fig.show()
pio.write_image(fig, f'images/{save_name}/subject_{phantom_choice}/t1_t2.jpeg')


FileNotFoundError: [Errno 2] No such file or directory: 'images\\mouse_mt_107\\subject_3\\t1_t2.jpeg'

## 1.2. Create quant maps ##

## MT maps (protocol mt52) ##

In [127]:
mt_quant_maps_fn = os.path.join('images', save_name, f'subject_{subject_i+1}', 'mt_quant_maps.mat')
mt_quant_maps = sio.loadmat(mt_quant_maps_fn)
mt_fs = mt_quant_maps['fs'] * 110e3 / 1
mt_ksw = mt_quant_maps['ksw']
mt_dp = mt_quant_maps['dp']
mt_t1w = mt_quant_maps['t1w'] * 1000
mt_t2w = mt_quant_maps['t2w'] * 1000


In [128]:
acquired_data_fn = os.path.join(glu_mouse_mrf_files_fn, 'acquired_data.mat')
acquired_data = sio.loadmat(acquired_data_fn)['acquired_data']  # [31, 40/64, 64]
# px.imshow(acquired_data[4,:,:])


full_mask = subject_dict['roi_mask']
date = subject_dict['month']
save_name = subject_dict['save_name']
temp = subject_dict['temp']
# f_lims = subject_dict['dict_ranges']['fs_0']
# k_lims = subject_dict['dict_ranges']['ksw_0']

# Create custom Viridis colormap with black for 0 values
custom_viridis = np.array(plotly.colors.sequential.Viridis)
custom_viridis[0] = '#000000'  # Set black for 0 values

# Create custom Plasma colormap with black for 0 values
custom_plasma = np.array(plotly.colors.sequential.Plasma)
custom_plasma[0] = '#000000'  # Set black for 0 values

# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.07, subplot_titles=['MT [mM]', 'ksw [Hz]', 'Dot Product'])

# Add heatmaps for the three arrays
heatmap_fs = go.Heatmap(z=mt_fs*full_mask, colorscale=custom_viridis, coloraxis='coloraxis1')
heatmap_ksw = go.Heatmap(z=mt_ksw*full_mask, colorscale=custom_plasma, coloraxis='coloraxis2')

fig.add_trace(heatmap_fs, row=1, col=1)
fig.add_trace(heatmap_ksw, row=1, col=2)

fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"Phantom {date} {temp}°C: '{fp_prtcl_name}' MT MRF results",
    showlegend=False,  # Hide legend
    height=345,
    width=900,  # Set a width based on your preference
    margin=dict(l=10, r=40, t=60, b=20),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)

# Add individual titles and separate colorbars
for i, title in enumerate(['Glu [mM]', 'ksw [Hz]', 'Dot Product'], start=1):
    fig.update_xaxes(row=1, col=i, showgrid=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, row=1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# Manually add separate colorbars
f_const = 3 / 110000
colorbar_fs = {'colorscale': custom_viridis} #, 'cmin': 0, 'cmax': f_lims[1]/f_const}
colorbar_ksw = {'colorscale': custom_plasma} #, 'cmin': k_lims[0], 'cmax': k_lims[1]}

fig.update_layout(
    coloraxis1=colorbar_fs,
    coloraxis2=colorbar_ksw,
    coloraxis_colorbar=dict(x=0.46, y=0.5),
    coloraxis2_colorbar=dict(x=1, y=0.5),
)

# Show the plot
fig.show()

pio.write_image(fig, f'images/{save_name}/subject_{phantom_choice}/MT52.jpeg')


In [21]:
quant_maps_fn = os.path.join('images', save_name, f'subject_{subject_i+1}', 'quant_maps.mat')
quant_maps = sio.loadmat(quant_maps_fn)
fs_array = quant_maps['fs'] * 110e3 / 3
ksw_array = quant_maps['ksw']
dp_array = quant_maps['dp']
t1w_array = quant_maps['t1w'] * 1000
t2w_array = quant_maps['t2w'] * 1000


FileNotFoundError: [Errno 2] No such file or directory: 'images\\mouse_mt_107\\subject_3\\quant_maps.mat'

In [22]:
acquired_data_fn = os.path.join(glu_mouse_mrf_files_fn, 'acquired_data.mat')
acquired_data = sio.loadmat(acquired_data_fn)['acquired_data']  # [31, 40/64, 64]
# px.imshow(acquired_data[4,:,:])


full_mask = subject_dict['roi_mask']
date = subject_dict['month']
save_name = subject_dict['save_name']
temp = subject_dict['temp']
# f_lims = subject_dict['dict_ranges']['fs_0']
# k_lims = subject_dict['dict_ranges']['ksw_0']

# Create custom Viridis colormap with black for 0 values
custom_viridis = np.array(plotly.colors.sequential.Viridis)
custom_viridis[0] = '#000000'  # Set black for 0 values

# Create custom Plasma colormap with black for 0 values
custom_plasma = np.array(plotly.colors.sequential.Plasma)
custom_plasma[0] = '#000000'  # Set black for 0 values

# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.07, subplot_titles=['Glu [mM]', 'ksw [Hz]', 'Dot Product'])

# Add heatmaps for the three arrays
heatmap_fs = go.Heatmap(z=fs_array*full_mask, colorscale=custom_viridis, coloraxis='coloraxis1')
heatmap_ksw = go.Heatmap(z=ksw_array*full_mask, colorscale=custom_plasma, coloraxis='coloraxis2')

fig.add_trace(heatmap_fs, row=1, col=1)
fig.add_trace(heatmap_ksw, row=1, col=2)

fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"Phantom {date} {temp}°C: '{fp_prtcl_name}' Glutamate MRF results",
    showlegend=False,  # Hide legend
    height=345,
    width=900,  # Set a width based on your preference
    margin=dict(l=10, r=40, t=60, b=20),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)

# Add individual titles and separate colorbars
for i, title in enumerate(['Glu [mM]', 'ksw [Hz]', 'Dot Product'], start=1):
    fig.update_xaxes(row=1, col=i, showgrid=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, row=1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# Manually add separate colorbars
f_const = 3 / 110000
colorbar_fs = {'colorscale': custom_viridis} #, 'cmin': 0, 'cmax': f_lims[1]/f_const}
colorbar_ksw = {'colorscale': custom_plasma} #, 'cmin': k_lims[0], 'cmax': k_lims[1]}

fig.update_layout(
    coloraxis1=colorbar_fs,
    coloraxis2=colorbar_ksw,
    coloraxis_colorbar=dict(x=0.46, y=0.5),
    coloraxis2_colorbar=dict(x=1, y=0.5),
)

# Show the plot
fig.show()

pio.write_image(fig, f'images/{save_name}/subject_{phantom_choice}/{fp_prtcl_name}.jpeg')


NameError: name 'fs_array' is not defined

## AAICD example - Redo with mask later ##

In [131]:
from my_funcs.cest_functions import z_spec_rearranger, offset_rearranger, wassr_b0_mapping, b0_correction

# z-spec for roi
def roi_z(subject_dict):
    # given:
    gyro_ratio_hz = 42.5764  # for H [Hz/uT]
    b0 = 7
    
    glu_phantom_fn = os.path.join(general_fn, 'scans', subject_dict['scan_name'],
                                      subject_dict['sub_name'])
    b1_names = subject_dict['z_b1s_names']
    roi_mask = subject_dict['roi_voxel_mask']
    
    # WASSR image
    wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name,
                                                                                      'WASSR')
    wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
    M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)
    
    wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
    offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    b0_map = wassr_b0_mapping(wassr_norm, mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz * b0)
    
    z_dict = {}
    for b1_i, b1_name in enumerate(b1_names):  # 0.7, 1.5, 2, 4, 6
        b1_dict = {}
        # z-spec
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn,
                                                                                                txt_file_name,
                                                                                                b1_name)  # (72/58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
        cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
        M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
        z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
    
        # offset vector
        offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
        offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    
        b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz)  # have not checked!
    
        roi_mask = roi_mask.astype(np.uint8)
        
        c_n = z_spec_norm.shape[0]
        mean_z_spectrum_not_cor = np.zeros([c_n])
        mean_z_spectrum = np.zeros([c_n])
        for c_i in range(c_n):
            [[mean_vial]], _ = cv2.meanStdDev(z_spec_norm[c_i, :, :], mask=roi_mask)  # before b0 correction
            mean_z_spectrum_not_cor[c_i] = mean_vial
            [[mean_vial]], _ = cv2.meanStdDev(b0_cor_zspec[c_i, :, :], mask=roi_mask)
            mean_z_spectrum[c_i] = mean_vial
        
        fr_s = int(bruker_dataset['SatFreqStart'].value)
        fr_e = int(bruker_dataset['SatFreqEnd'].value)
        fr_i = round(bruker_dataset['SatFreqInc'].value, 2)
    
        ppm = np.arange(fr_s, fr_e-fr_i, -fr_i)
    
        b1_dict['ppm'] = ppm
        b1_dict[f'z_b1cor'] = mean_z_spectrum
        b1_dict[f'z_nob1cor'] = mean_z_spectrum_not_cor
        z_dict[f'{b1_name}'] = b1_dict
        
    return z_dict

z_dict = roi_z(subject_dict)


WASSR B0 mapping took 1.203 seconds


In [132]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import splev, splrep

# given:
z_dict = roi_z(subject_dict)
save_name = subject_dict['save_name']
month = subject_dict['month']
temp = subject_dict['temp']
b1_names = subject_dict['z_b1s_names']
b1s = subject_dict['z_b1s']
b1s = [f'{x}uT' for x in b1s]

# mean_z_spectrum_not_cor, mean_z_spectrum = z_spectra_creator(general_fn, txt_file_name, subject_dict)
# 
# cest_prtcl_names = subject_dict['z_b1s_names']
# tag = subject_dict['tags']
# conc_l = subject_dict['concs']
# ph_l = subject_dict['phs']

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

fig = make_subplots(rows=1, cols=len(b1_names),
                    subplot_titles=b1s, vertical_spacing=0.07, horizontal_spacing=0.01)
fig_2 = make_subplots(rows=1, cols=len(b1_names),
                      subplot_titles=b1s, vertical_spacing=0.07, horizontal_spacing=0.05)

# Iterate over each channel
for b1_i, b1_name in enumerate(b1_names):
    # Iterate over each image
    ppm = z_dict[b1_name]['ppm']
    z_b1cor = z_dict[b1_name]['z_b1cor']
    z_nob1cor = z_dict[b1_name]['z_nob1cor']

    # MTR asym calculation
    cha_n = len(z_b1cor)
    mid_i = int(cha_n / 2 - 1)
    last_i = int(cha_n - 1)
    positives = z_b1cor[0:(mid_i + 2)]  # positives to 0 (including)
    negatives = z_b1cor[last_i:mid_i:-1]  # negatives to 0 (including)

    MTR_asym = negatives - positives
    ppm_asym = np.linspace(ppm[0], 0, num=(mid_i + 2))
    
    if b1_name == '1p5uT':
        MainFieldMHz = 298
        B1_uT = 1.5
        gyro_ratio_hz = 42.5764
        w1_Hz = round(B1_uT*gyro_ratio_hz)  # satpwr for scan (Hz)

        w_Hz = ppm * MainFieldMHz  # Scanned frequency offsets (ppm * MHz = Hz)

        Interp_w_Hz = np.arange(min(w_Hz), max(w_Hz)+1, 1)  # Spline interpolation of scanned offsets every 1 Hz
        tck = splrep(w_Hz[::-1], z_b1cor[::-1], s=0)  # Spline representation of the data (I needed to reverse)
        M_2p75 = splev(2.75*MainFieldMHz, tck)
        M_3p5 = splev(3.5*MainFieldMHz, tck)

        M_6 = z_b1cor[np.argmin(abs(ppm-6))]
        AACID = (M_3p5 * (M_6 - M_2p75)) / (M_2p75 * (M_6 - M_3p5))
        # ph = (AACID - 6.5) / (-0.64)
        ph = -4*AACID + 12.8
        
        print(f'AACID is: {AACID:.2f}, extracted ph: {ph:.2f}')

    col_idx = b1_i + 1
    fig.add_trace(go.Scatter(x=ppm, y=z_b1cor, line=dict(color=colors[0]), legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
    # fig.add_trace(go.Scatter(x=ppm, y=z_vial_B_not_cor, line=dict(color='gray'), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)  # later delete!

    fig.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[0]), legendgroup=f'{1}_{col_idx}_2'), row=1,
                  col=col_idx)
    fig.update_xaxes(title_text='ppm', row=1, col=col_idx)

    fig_2.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[0]), legendgroup=f'{1}_{col_idx}_2'), row=1,
                    col=col_idx)
    fig_2.update_xaxes(title_text='ppm', row=1, col=col_idx)

fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
fig_2.update_yaxes(title_text='MTR-asym', row=1, col=1)

# Update layout
fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Phantom {month} - {temp}°C Z-spectrum',
                  height=250, width=250*len(b1_names)+150,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=45, r=0, t=50, b=0)
                  )  # Adjust the title position

fig_2.update_layout(template='plotly_white',  # Set the theme to plotly white
                    title_text=f'Phantom {month} - {temp}°C MTR-asym',
                    height=200, width=220*len(b1_names)+150,
                    title=dict(x=0.02, y=0.97),
                    margin=dict(l=0, r=0, t=60, b=0)
                    )  # Adjust the title position

# Set axes for z-spectrum
fig.update_xaxes(autorange='reversed', tickmode='linear', tick0=0, dtick=1)
fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])

# # Set axes for MTR-asym
fig_2.update_xaxes(autorange='reversed', dtick=1)
# fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
fig_2.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[0, 0.25], tickformat='.0%')

# Show only specific legend groups
for trace in fig.data:
    trace.showlegend = (trace.legendgroup == '3')

# Show only specific legend groups
for trace in fig_2.data:
    trace.showlegend = (trace.legendgroup == '3')

fig.show()
fig_2.show()

pio.write_image(fig, f'images/{save_name}/subject_{phantom_choice}/z_spec/zspec.jpeg')
pio.write_image(fig_2, f'images/{save_name}/subject_{phantom_choice}/z_spec/mtr.jpeg')


WASSR B0 mapping took 1.185 seconds
AACID is: 1.37, extracted ph: 7.32
